In [27]:
import numpy as np

### Initialization

In [34]:
def random_initialization(layer_dims):
    """
    Arguments:
    layer_dims -- list with the dimensions of each layer in our network
    
    Performs random initialization for the weight matrices and zero initialization for biases.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

In [ ]:
def xavier_initialization(layer_dims):
    pass

In [ ]:
def he_initialization(layer_dims):
    pass

### Regularization

In [ ]:
def get_L2_regularization_cost(parameters, lambd):
    """
    Return L2 regularization term.
    
    Arguments:
    parameters -- python dictionary containing parameters of the model
    lambd -- regularization hyperparameter, scalar

    Returns:
    cost - value of the regularization term to be used for cost function regularization
    """
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]

    sum_of_squares = np.sum(np.square(W1) + np.sum(np.square(W2)) + np.sum(np.square(W3)))
    L2_regularization_cost = 1/m * lambd/2 * sum_of_squares
    
    return L2_regularization_cost

### Mini-batch

In [9]:
def shuffle(vec, num_examples):
    if len(vec) == 0:
        return vec
    permutation = list(np.random.permutation(num_examples))
    return vec[:, permutation]

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, shape (input size, number of examples)
    Y -- ground thruth vector, shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(seed)
    m = X.shape[1]
    mini_batches = []
        
    shuffled_X = shuffle(X)
    shuffled_Y = shuffle(Y).reshape((1,m))

    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : (k+1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : (k+1) * mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        num_remaining_examples = m - mini_batch_size * np.floor(m / mini_batch_size)
        last_example_index = num_complete_minibatches * mini_batch_size + num_remaining_examples
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : last_example_index]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : last_example_index]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


### Optimization algorithms

In [59]:
def relu(z):
    x = np.array(z, copy=True)
    np.maximum(x, 0, x) # modifies x in place
    return x

In [62]:
# Test relu
print ("relu([0, 2]) = " + str(relu(np.array([0,2]))))

relu([0, 2]) = [0 2]


In [54]:
def sigmoid(z):
    return  1.0 / (1 + np.exp(-z))

In [63]:
# Test sigmoid
print("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))
print("exptected: [ 0.5 0.88079708]") 

sigmoid([0, 2]) = [ 0.5         0.88079708]
exptected: [ 0.5 0.88079708]


In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    else:
        raise Error("Activation function " + activation + " is not supported.")
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [42]:
def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2 # of layers in the network
    
    # Implement [LINEAR -> RELU]*(L-1)
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], "relu")
        caches.append(cache)
    
    # Implement LINEAR -> SIGMOID
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [40]:
def compute_cost(A_last, Y):
    """
    Implement the cost function with L2 regularization. See formula (2) above.
    
    Arguments:
    A_last -- post-activation, output of forward propagation, of shape (output size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    
    Returns:
    cost - value of the loss function
    """
    m = Y.shape[1]
    cost = - 1.0 / m * np.sum((np.multiply(np.log(A_last), Y) + np.multiply(np.log(1 - A_last), 1 - Y)))
    
    return cost

In [41]:
# Test compute_cost()
A_last = np.array([[0.40682402, 0.01629284, 0.16722898, 0.10118111, 0.40682402]])
Y = np.array([[1, 1, 0, 1, 0]])

assert(compute_cost(A_last, Y) - 1.60250160476 < 10e-5)

In [24]:
def gradient_descent(X, Y, seed = 0):
    pass

### Neural network model

In [ ]:
def nn_model(X, Y, n_h, layers_dims, learning_rate = 0.0075, num_iterations = 100, mini_batch_size = 64,
             seed = 0, print_cost=False):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    """
    n_x = X.shape[0]
    n_y = Y.shape[0]
    n_h = 4
    
    np.random.seed(1)
    costs = []

    parameters = random_initialization(layers_dims)

    for i in range(0, num_iterations):
        A_last, cache = L_model_forward(X, parameters)
        cost = compute_cost(A_last, Y)
        grads = L_model_backward(A_last, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Print cost
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters